In [367]:
import pandas as pd
import requests


In [368]:
crop_csv = pd.read_csv('crop.csv')
soil_csv = pd.read_csv('soil.csv')
spectral_csv = pd.read_csv('spectral.csv')
weather_csv = pd.read_csv('weather.csv')

In [369]:
#challenge_1 

crop_1 = crop_csv[crop_csv["year"] == 2021]
crop_2 = crop_1[["field_id", "field_geometry", "crop_type"]]
crop_2.to_csv('Transformed_crop.csv', index = False)

In [370]:
#challenge_2

spectral_csv['NDVI'] = (spectral_csv.nir - spectral_csv.red) / (spectral_csv.nir + spectral_csv.red)
spectral_csv_2 = spectral_csv[spectral_csv.NDVI == spectral_csv.NDVI.max()]
spectral_csv_2 = spectral_csv_2.rename(columns={"NDVI": "pos", "date": "pos_date"})
spectral_csv_3 = spectral_csv_2[['tile_id', 'tile_geometry', 'pos', 'pos_date']]
spectral_csv_3.to_csv('Transformed_spectral.csv', index = False)

In [371]:
#challenge_3

soil_csv['avg_horizontal_layer'] = (abs(soil_csv.hzdept - soil_csv.hzdepb))/soil_csv.hzdepb

soil_csv['avg_components_map_unit'] = (soil_csv.comppct *  soil_csv.avg_horizontal_layer)/ 100

soil_csv_2 = soil_csv[['mukey', 'mukey_geometry', 'om', 'cec', 'ph', 'avg_horizontal_layer', 'avg_components_map_unit']]

soil_csv_2.to_csv('Transformed_soil.csv', index = False)

In [372]:
#challenge_4

weather_csv_2 = weather_csv[(weather_csv["year"] == 2021)]

weather_csv_3 = weather_csv_2.groupby('fips_code').agg({'temp': ['mean', 'min', 'max', 'sum']})
weather_csv_3.columns = weather_csv_3.columns.droplevel()
cropped_crop_csv = crop_csv[['field_id', 'field_geometry']]
cropped_crop_csv_2 = cropped_crop_csv.drop_duplicates(subset=['field_id'])
cropped_crop_csv_3 = cropped_crop_csv_2['field_geometry'].str.split('MULTIPOLYGON', expand=True)

splited_coordinates = [cropped_crop_csv_3[1].str.split(',', expand=True)]

splited_coordinates = splited_coordinates[0][1]

state = []
county = []
for i in splited_coordinates:
  i = i.split(' ')
  url = 'https://geo.fcc.gov/api/census/block/find?latitude=%s&longitude=%s&format=json' % (i[1],i[0])
  response = requests.get(url)
  data = response.json()
  state.append(data['State']['FIPS'])
  #county.append(data['County']['FIPS'][2:])
  county.append(data['County']['FIPS'])
  
cropped_crop_csv_2['fips_code'] = county


cropped_crop_csv_2['fips_code'] = cropped_crop_csv_2["fips_code"].astype(int)

cropped_crop_csv_4 = pd.merge(weather_csv_3, cropped_crop_csv_2, on="fips_code")

cropped_crop_csv_5 = cropped_crop_csv_4[['field_id','mean', 'min',	'max',	'sum']]
cropped_crop_csv_5.to_csv('Transformed_weather.csv', index = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
